In [1]:
# Add techlock package without installing it
import os.path
import sys
sys.path.insert(1, os.path.abspath('../'))

import psycopg2
from psycopg2.extensions import cursor as cursor_type
psql_host = '192.168.124.10'
src_psql_port = 5434
dest_psql_port = 6104

src_conn = psycopg2.connect(
  host=psql_host,
  port=src_psql_port,
  user="postgres",
  password="postgres",
  database="auditplatform",
)

dest_conn = psycopg2.connect(
  host=psql_host,
  port=dest_psql_port,
  user="techlock",
  password="password",
  database="postgres",
)

In [2]:
tenant_id_mapping = {
  92: 'a6fe7042-abcd-4f77-ae07-ee49328f2ecf',     # ANN  - Annuity Health
  82: '2a94a170-f97d-4ce9-a6b0-02bbb639bb3f',     # APS  - APS Medical
  38: '52321d13-c8f2-452f-bfb2-16e3e01ae840',     # ARA  - AR Audit Services
  73: '4bdb0ff1-6e1d-429e-afab-be200e1b77e1',     # ARM  - Armada Labs
  43: '7f4f3173-0e09-44d4-8961-84676fa96991',     # ARS  - Asset Recovery Solutions
  68: '9ca1377e-3c27-48cd-89ca-2bb2c1cdcc4d',     # BCA  - BCA Financial Services, INC
  76: 'e050642c-03a9-43ff-bb02-589921f6bac2',     # BCS  - BC Services, Inc.
  67: 'a9465e49-2a8a-48de-b1cd-315a83323c01',     # BHR  - Balanced Healthcare Receivables
  91: '811ea325-d0f8-42b3-b01d-9c1087b8f0b6',     # BAS  - Black Angus Steakhouses LLC
  54: '87971662-951d-408f-b20c-d0746fb2acdb',     # CAI  - Credit Adjustments Inc
  72: '53d4b2b3-9d9d-46b7-9080-91909e4a8277',     # CAS  - Cascade
  79: '3384a8e7-e6e4-465c-878b-15b34238d3fb',     # CEN  - Centrada
  61: '71712fc1-d4c9-47d5-b0eb-419a59d19cd2',     # CMI  - The CMI Group
  84: '608a3e63-4ac8-4597-8afb-ff753682d994',     # CMW  - Commonwealth Financial Systems
  62: '37ab7790-da21-4c69-9167-889d2173c935',     # COH  - Coalesce Payments
  60: 'ff526b89-b0bf-4969-a708-19cd384d16da',     # CRI  - Central Research
  59: '4a811b1f-cbf8-44d6-b73f-37e4d91d1fc4',     # CSS  - Clarity Software Solution
  56: '22ca1c17-c6ec-4b88-8c53-1df1fc82442a',     # IDB  - Indebted Co (Delta Outsource Group)
  53: '740b245c-94c4-4902-bb41-f84ab7152455',     # DTE  - DTE Energy
  51: '01127764-e6f0-4bb7-8cfd-082224f57115',     # FAST - Frontline Asset Startegies
  88: '39b57444-954a-49bb-a30d-c845111196f0',     # GCC  - Gulf Coast Collection Bureau Inc
  39: '91c9e31d-8e66-4dfa-8e8c-58c4fca7704a',     # HAR  - Harris & Harris
  37: '5380333b-3154-41d6-bc31-62fa75261400',     # HIN  - Hinda
  35: 'c747d0ef-4cd9-4392-9626-4cbb936ac2ab',     # FCS - First Collection Services
  89: '4d6216e6-48aa-4ea1-a4c4-3ccb2d6d3eee',     # MCG  - Merchants Credit Guide
  85: '266b1d2f-fd0e-43be-80a4-ee020501a143',     # MED  - Meduit
  32: 'c7c6df09-308a-40c7-9c6d-d7192f893f74',     # NCS  - National Credit Service
  29: 'ace86bf7-473d-45d4-884c-f288e1c881b5',     # NSLP - National Student Loan Program
  27: 'dca7c705-13df-4ab3-b5b5-f53fc334a251',     # OLM  - Olmstead Medical Center
  83: '4bdb0ff1-6e1d-429e-afab-be200e1b77e1',     # PAY  - Payline
  25: '0118ea10-7c67-48f0-aabe-c4acc0c391fd',     # PHN  - Phin Solutions
  86: '4af2f248-f63c-40c2-b3c3-be95c54bf330',     # PRO  - Pro Asset Information Solutions
  21: '8f1f9007-cd7f-4404-a3d2-ae34c1395ebd',     # RGS  - RGS Financial
  19: 'eafaad26-aca0-4b04-a100-c96332ce36fa',     # RNN  - The RNN Group
  87: '6f9dfdb9-5ca7-4d0c-be18-38f3bca1bd72',     # RVC  - Revco Solutions
  17: 'a25966e4-3228-407b-a659-3bc7b75827b4',     # SYG - Synergetic Communications
  2 : 'ec2f47d9-d142-467a-b0bb-94e270278361',     # SCS  - State Collection Services
  15: '74fe5a9a-d227-4587-830f-4df94e137507',     # SPS  - Stratus Payment Solutions
  12: 'e76ee26f-41ca-4249-940f-da1a5babe9d6',     # TEC  - TEC Service Service Group
  11: '2f7f53dc-81af-4b3a-8c5b-fe52046774f4',     # TLC  - Tech Lock Inc
  90: 'fe71e547-4542-490f-acd6-edeede135a4a',     # USC  - USCB America
  8 : '8de0e34c-4cc2-4369-b8ce-e0caf3f36357',     # VER  - Verifacts
  5 : 'ece54005-b486-44b2-85bb-76f62f8555c7',     # WIL  - Williams & Fudge
  4 : '65a09af0-1bc5-4a69-b3ff-150d0a3e9371',     # WIN  - Windham Professionals
}


In [17]:
import os
import pandas as pd
from dataclasses import dataclass
from psycopg2.extensions import cursor as cursor_type
from typing import Any, Dict, List, Union, Callable
from auth0.v3.authentication import GetToken
from auth0.v3.management import Auth0
from environs import Env
from pprint import pprint

from techlock.compass.models.audit import Phase as AuditPhase

Env().read_env('./')

domain = 'techlock.auth0.com'
audience = 'https://techlock.auth0.com/api/v2/'
connection_id = 'MSS-PROD'
resource_server_identifier = 'PROD_COMPASS'

## Create a `.env` file with these variables
client_id = os.getenv('client_id')
client_secret = os.getenv('client_secret')
if not (client_id and client_secret):
  raise Exception('No client_id or client_secret detected!')

get_token = GetToken(domain)
token = get_token.client_credentials(client_id, client_secret, audience)
mgmt_api_token = token['access_token']
auth0 = Auth0(domain, mgmt_api_token)

base_columns = [
  'tenant_id',
  'created_by',
  'created_on',
  'changed_by',
  'changed_on',
  'name',
  'description',
  'tags',
  'is_active',
]

base_column_mapping = {
  'tenant_id': lambda _: tenant_id_mapping[client_id],
  'created_by': lambda row: get_user(row.get('user_id')),
  'changed_by': lambda row: get_user(row.get('user_id')),
  'created_on': 'inserted_at',
  'changed_on': 'updated_at',
  'is_active': lambda row: not row.get('deleted'),
}


user_mapping = {}
def get_user(user_id):
  try:
    if user_id not in user_mapping:
      user = auth0.users.get(user_id)
      user_mapping[user_id] = user.get('email')

    user_email = user_mapping.get(user_id)
  except Exception as e:
    user_email = None
    print(f'Failed to find user by id {user_id} - {e}')

  return user_email


@dataclass
class Copier:
  src_cursor: cursor_type
  dest_cursor: cursor_type

  def get_old_data(
    self,
    client_id: int,
    table: str,
    columns: List[str],
  ):
    self.src_cursor.execute(f'''
      SELECT {','.join(f'"{c}"' for c in columns)}
      FROM client_{client_id}.{table}
    ''')
    rows = self.src_cursor.fetchall()
    
    print(f' - For {client_id} - {table} - {len(rows)} rows')
    return rows

  def process_rows(
    self,
    old_rows: List[Any],
    old_columns: List[str],
    new_columns: List[str],
    column_mapping: Dict[str, Union[str, Callable]],
  ):
    new_rows = []
    for old_row in old_rows:
      # add columns names
      old_row = dict(zip(old_columns, old_row))

      new_row = []
      # skip entity_id column as it's auto generated
      for column in new_columns:
        src_column = column_mapping.get(column, column)
        if callable(src_column):
          value = src_column(old_row)
        else:
          value = old_row.get(src_column)
        
        new_row.append(value)

      new_rows.append(new_row)

    return new_rows

  def insert_new_data(
    self,
    table: str,
    columns: List[str],
    new_values: List[List[Any]],
  ) -> List[int]:
    if not new_values:
      return []

    new_values_str = b','.join(
      self.dest_cursor.mogrify(
        # create '(%s,%s)' for number of columns
        f"({','.join('%s' for _ in columns)})",
        row
      ) for row in new_values
    ).decode('utf8')

    self.dest_cursor.execute(f'''
      INSERT INTO {table}
      ({','.join(f'"{c}"' for c in columns)})
      VALUES {new_values_str}
      RETURNING entity_id
    ''')
    new_ids = self.dest_cursor.fetchall()
    return [x[0] for x in new_ids]

  def copy(
    self,
    client_id: int,
    table: str,
    old_columns: List[str],
    new_columns: List[str],
    column_mapping: Dict[str, Union[str, Callable]],
  ):
    # Get old rows
    old_rows = self.get_old_data(client_id, table, old_columns)
    old_ids = [r[0] for r in old_rows]
    # Generate new rows from the old ones
    new_rows = self.process_rows(old_rows, old_columns, new_columns, column_mapping)
    # Insert new rows, and return the new ids
    new_ids = self.insert_new_data(table, new_columns, new_rows)
    # Map old ids to new ids
    old_to_new_id = dict(zip(old_ids, new_ids))

    return old_to_new_id

  def copy_audits(self, client_id: int):
    table = 'audits'
    # columns of the old table to get
    old_columns = [
      "id",
      "user_id",
      "name",
      "reports",
      "start_date",
      "estimated_remediation_date",
      "remediation_date",
      "estimated_end_date",
      "end_date",
      "phase",
      "deleted",
      "inserted_at",
      "updated_at",
    ]
    # New columns to insert
    new_columns = [
      *base_columns,
      'auditor',
      'reports',
      'start_date',
      "estimated_remediation_date",
      "remediation_date",
      "estimated_end_date",
      "end_date",
      "phase",
    ]
    # Map the new columns to how to retrieve them
    # Key is the new column name
    # Value is either a string if it's a simple rename, 
    #   or callable if we need to do something more complex.
    column_mapping = {
      **base_column_mapping,
      'auditor': lambda row: get_user(row['user_id']),
      'phase': lambda row: AuditPhase(row['phase']).name,
    }

    old_to_new_id = self.copy(client_id, table, old_columns, new_columns, column_mapping)

    return old_to_new_id

  def copy_audit_responses(self, client_id: int, audit_ids: Dict[int,int]):
    table = 'audit_responses'
    old_columns = [
      "id",
      "audit_id",
      "instruction_id",
      "text",
      "compliance",
      "inserted_at",
      "updated_at",
    ]
    new_columns = [
      *base_columns,
      "audit_id",
      "instruction_id",
      "compliance",
    ]

    old_rows = self.get_old_data(client_id, table, old_columns)
    old_ids = [r[0] for r in old_rows]
    
    column_mapping = {
      **base_column_mapping,
      'description': lambda row: get_user(row['text']),
      'audit_id': lambda row: old_ids[row['audit_id']],
      'instruction_id': lambda row: 'TODO!!'
    }
    
    new_rows = self.process_rows(old_rows, old_columns, new_columns, column_mapping)
    new_ids = self.insert_new_data(table, new_columns, new_rows)
    old_to_new_id = dict(zip(old_ids, new_ids))

    return old_to_new_id


with src_conn:
  with src_conn.cursor() as src_cursor:
    src_cursor.execute('''
      SELECT schema_name 
      FROM information_schema.schemata
      WHERE schema_name LIKE 'client_%'
    ''')
    rows = src_cursor.fetchall()

    client_ids = [int(row[0][7:]) for row in rows]

    for client_id in client_ids:
      copier = Copier(src_cursor)
      audits = copier.copy_audits(client_id)

      df = pd.DataFrame(audits)
      break

df

 - For 92 - audits - 4 rows


,tenant_id,created_by,created_on,changed_by,changed_on,name,description,tags,is_active,auditor,reports,start_date,estimated_remediation_date,remediation_date,estimated_end_date,end_date,phase
0,a6fe7042-abcd-4f77-ae07-ee49328f2ecf,jluke@techlockinc.com,2019-07-12 14:04:25,jluke@techlockinc.com,2019-11-11 21:10:41,2019-RMD-TLC,None,None,True,jluke@techlockinc.com,"[3, 6, 9, 12, 14, 19, 18]",2019-07-12,2019-09-08,2019-11-11,2019-12-06,2019-11-11,billing_and_marketing
1,a6fe7042-abcd-4f77-ae07-ee49328f2ecf,jluke@techlockinc.com,2020-09-08 11:49:58,jluke@techlockinc.com,2020-09-22 00:00:01,2020-RMD-TLC,None,None,True,jluke@techlockinc.com,"[6, 9, 12, 14, 19, 18]",2020-09-08,2020-11-05,None,2021-02-02,None,scoping_and_validation
2,a6fe7042-abcd-4f77-ae07-ee49328f2ecf,jluke@techlockinc.com,2021-10-26 16:46:42,jluke@techlockinc.com,2021-11-09 00:00:03,2021-RMD-TLC,None,None,True,jluke@techlockinc.com,"[3, 6, 12, 14, 19, 18]",2021-10-26,2021-12-23,None,2022-03-22,None,scoping_and_validation
3,a6fe7042-abcd-4f77-ae07-ee49328f2ecf,ngalloway@techlockinc.com,2018-04-17 14:19:31,ngalloway@techlockinc.com,2018-09-06 12:06:22,2018-RMD-TLC,None,None,True,ngalloway@techlockinc.com,"[3, 6, 9, 12, 14, 17, 18]",2018-03-19,2018-05-16,2018-05-18,2018-08-13,2018-09-04,billing_and_marketing
